In [16]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [18]:
# File path of the input Excel
file_path = "Kadam Haryana compile report  2024-2025 02052025.xlsx"

# Load the data sheet into pandas DataFrame
data_df = pd.read_excel(file_path, sheet_name='Compile Report')

# Define validation rules in the script as a dictionary
# Format: column_name: list of validations to apply
# Supported validations: "not null", "numeric", "date"
validation_rules = {
    "Student's First Name": ["not null"],
    "Student's Last Name": ["not null"],
    "Student's Age": ["not null", "numeric"],
    "Student's Date of Birth": ["date"],
    # Add your columns and validation rules here as needed
}

ERROR! Session/line number was not unique in database. History logging moved to new session 64


In [19]:
# Define red fill for errors in Excel
red_fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

# Load workbook and worksheet for applying Excel formatting
wb = load_workbook(file_path)
ws = wb['Compile Report']

# List to hold validation error records
validation_errors = []

In [20]:
# Iterate over the validation_rules dictionary
for col_name, rules in validation_rules.items():
    if col_name not in data_df.columns:
        # Column not in data, skip or optionally log warning
        continue

    # Find the Excel column letter for the column header
    col_letter = None
    for cell in ws[1]:
        if cell.value == col_name:
            col_letter = cell.column_letter
            break
    if col_letter is None:
        # Could not find the column in Excel, skip
        continue

    # Iterate over each value in the column
    for i, value in enumerate(data_df[col_name], start=2):  # Excel rows start at 2 due to header
        for rule in rules:
            has_error = False
            error_reason = ""
            rule_lower = rule.lower()

            if rule_lower == "not null":
                if pd.isnull(value) or (isinstance(value, str) and value.strip() == ""):
                    has_error = True
                    error_reason = "Value is null or empty"
            elif rule_lower == "numeric":
                try:
                    float(value)
                except Exception:
                    has_error = True
                    error_reason = "Value is not numeric"
            elif rule_lower == "date":
                if pd.to_datetime(value, errors='coerce') is pd.NaT:
                    has_error = True
                    error_reason = "Value is not a valid date"
            # Add additional rules here as desired

            if has_error:
                cell_ref = f"{col_letter}{i}"
                ws[cell_ref].fill = red_fill
                validation_errors.append({
                    "Row": i,
                    "Column": col_name,
                    "Cell": cell_ref,
                    "Value": value,
                    "Error": error_reason
                })
                # Once error found for this rule in this cell, no need to check other rules for same cell
                break

C:\Users\nehak\AppData\Local\Temp\ipykernel_4884\415948242.py:35: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  if pd.to_datetime(value, errors='coerce') is pd.NaT:


In [21]:
# Save the highlighted workbook as new file
wb.save("Validated_Output.xlsx")

# Create a DataFrame for validation report
report_df = pd.DataFrame(validation_errors)

# Save the validation report as a separate Excel file (empty if no errors)
if report_df.empty:
    report_df = pd.DataFrame(columns=["Row", "Column", "Cell", "Value", "Error"])

report_df.to_excel("Validation_Report.xlsx", index=False)

print("Validation complete.")
print("Highlighted data saved as 'Validated_Output.xlsx'.")
print("Validation report saved as 'Validation_Report.xlsx'.")

Validation complete.
Highlighted data saved as 'Validated_Output.xlsx'.
Validation report saved as 'Validation_Report.xlsx'.
